In [10]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import tensorflow as tf

In [5]:
train_data = pd.read_csv('data/train.csv')
X_train = train_data.iloc[:, 4:-1].to_numpy()
Y_train = train_data.iloc[:,-1].to_numpy()
X_train, Y_train = shuffle(X_train, Y_train)
train_data

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA
0,1643,18,1,Caucasian,Higher,18.118879,24,1,NaN,1,1,0,0,1.396278
1,2753,16,1,Other,Some College,2.469271,19,0,Low,0,0,0,1,0.754751
2,2402,17,0,Asian,Bachelor,3.456401,25,0,NaN,0,0,0,0,0.136166
3,3033,17,1,Caucasian,High School,15.838131,25,1,Moderate,0,0,0,0,1.172192
4,1991,16,0,Caucasian,NaN,1.045373,19,1,Moderate,0,0,1,0,1.747356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2639,15,0,African American,NaN,13.092424,13,1,Low,0,0,0,0,1.595627
1909,2096,16,0,Caucasian,Some College,2.159249,19,0,Very High,0,1,0,0,1.523945
1910,2131,16,1,Caucasian,Some College,3.572578,14,1,Moderate,1,1,0,0,2.051919
1911,2295,18,0,Other,Some College,4.402585,20,0,High,0,0,0,0,1.348997


In [11]:
def one_hard_encoding(data):
    classes = set(data)
    class_dict = {}
    N = len(data)
    K = len(classes)
    ohe = np.zeros((N, K))
    for i, clas in zip(range(K), classes):
        class_dict[clas] = i
    for i in range(N):
        ohe[i, class_dict[data[i]]] = 1
    return ohe
    

In [14]:
X_train = np.concatenate([one_hard_encoding(X_train[:, 0]), X_train[:, 1:4], one_hard_encoding(X_train[:, 4]), X_train[:, 5:]], axis=1).astype(np.float32)

In [16]:
X_train.shape

(1913, 21)

In [9]:
class H_layer(object):
    def __init__(self, M1, M2):
        self.M1 = M1
        self.M2 = M2
        W = np.random.randn(M1, M2) / np.sqrt(M1)
        b = np.zeros(M2)
        self.W = tf.Variable(W.astype(np.float32))
        self.b = tf.Variable(b.astype(np.float32))
        self.params = [self.W, self.b]

    def forward(self, X):
        return tf.nn.relu(tf.matmul(X, self.W) + self.b)

In [ ]:
class ANN(object):
    def __init__(self, hidden_layer_sizes, p_keep):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.p_keep = p_keep

    def fit(self, X, Y, lr=1e-3, mu=0.99, decay=0.999, epochs=300, batch_sz=100, split=True, print_every=20):
        N, D = X.shape
        K = 1
        self.hidden_layer = []
        M1 = D
        for M2 in self.hidden_layer_sizes:
            self.hidden_layer.append(H_layer(M1, M2))
            M1 = M2

        W = np.random.randn(M1, K) / np.sqrt(M1)
        b = np.zeros(K)
        self.W = tf.Variable(W.astype(np.float32))
        self.b = tf.Variable(b.astype(np.float32))

        self.params = [self.W, self.b]
        for h in self.hidden_layer:
            self.params += h.params

        inputs = tf.placeholder(tf.float32, shape=(None, D), name="input")
        labels = tf.placeholder(tf.int64, shape=(None,), name="labels")
        logits = self.forward(inputs)

        

